# Convolutional Neural Networks For Image Recognition

## Setup and Imports

In [1]:
# Setup
from __future__ import division, print_function, unicode_literals

# Imports
import numpy as np
import numpy.random as rnd
import os

# To make make consistent across code blocks
rnd.seed(42)

# Ploting
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Saving Parameters
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "data", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

**Import Keras**

In [2]:
import keras as kr

Using TensorFlow backend.


# Convolutional Neural Network

**Load Coarse-To-Fine Mappings**

In [3]:
# Get Coarse to Fine Mappings
C2K = np.loadtxt('C2K.txt', dtype=int)
C2K_dot = np.zeros((20,100))
for i in range(len(C2K)):
    C2K_dot[C2K[i], i] = 1

#np.set_printoptions(threshold=np.inf)
#print(C2K_dot)

**Import Dataset**

In [4]:
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')

In [5]:
def to_one_hot(X):
    n_values = np.max(X) + 1
    y = np.eye(n_values)[X[:,0]]
    return y

y_train_adj = to_one_hot(y_train)
y_test_adj  = to_one_hot(y_test)

In [6]:
print(np.shape(x_train))

(50000, 32, 32, 3)


** Defining Structure**

In [7]:
def indep_layers(X):
    net = Conv2D(6, 2, strides=1, activation='elu', padding='same')(X)
    net = Dropout(0.25)(net)
    net = Conv2D(6, 2, strides=1, activation='elu', padding='same')(net)
    net = Dropout(0.25)(net)
    net = Conv2D(6, 2, strides=1, activation='elu', padding='same')(net)
    net = Flatten()(net)

    net = Dense(256, activation='elu')(net)
    net = Dropout(0.50)(net)
    net = Dense(128, activation='elu')(net)
    net = Dropout(0.25)(net)
    net = Dense(100, activation='softmax')(net)
    
    return net

In [8]:
from __future__ import absolute_import

from keras.layers import initializers
from keras.layers import regularizers
from keras.layers import constraints
from keras.engine.topology import Layer
from keras.engine.topology import InputSpec
from keras import backend as K
from keras.legacy import interfaces

class Threshold(Layer):
    """Thresholded Rectified Linear Unit.
    It follows:
    `f(x) = 1 for x > theta`,
    `f(x) = 0 otherwise`.
    # Input shape
        Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
    # Output shape
        Same shape as the input.
    # Arguments
        theta: float >= 0. Threshold value.
    """

    def __init__(self, theta=.5, **kwargs):
        super(Threshold, self).__init__(**kwargs)
        self.theta = K.cast_to_floatx(theta)

    def call(self, inputs, mask=None):
        return K.cast(K.greater(inputs, self.theta), K.floatx())

    def get_config(self):
        config = {'theta': float(self.theta)}
        base_config = super(Threshold, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [9]:
def calc_inputs():
    return [Input(shape=([100]), dtype='float32', name='c0'), \
        Input(shape=([100]), dtype='float32', name='c1'), \
        Input(shape=([100]), dtype='float32', name='c2'), \
        Input(shape=([100]), dtype='float32', name='c3'), \
        Input(shape=([100]), dtype='float32', name='c4'), \
        Input(shape=([100]), dtype='float32', name='c5'), \
        Input(shape=([100]), dtype='float32', name='c6'), \
        Input(shape=([100]), dtype='float32', name='c7'), \
        Input(shape=([100]), dtype='float32', name='c8'), \
        Input(shape=([100]), dtype='float32', name='c9'), \
        Input(shape=([100]), dtype='float32', name='c10'), \
        Input(shape=([100]), dtype='float32', name='c11'), \
        Input(shape=([100]), dtype='float32', name='c12'), \
        Input(shape=([100]), dtype='float32', name='c13'), \
        Input(shape=([100]), dtype='float32', name='c14'), \
        Input(shape=([100]), dtype='float32', name='c15'), \
        Input(shape=([100]), dtype='float32', name='c16'), \
        Input(shape=([100]), dtype='float32', name='c17'), \
        Input(shape=([100]), dtype='float32', name='c18'), \
        Input(shape=([100]), dtype='float32', name='c19')]

In [10]:
def calc_prob_ave(fine_pred, fine_to_coarse):
    coarse_pred = Dot(1)([fine_pred,fine_to_coarse])
    coarse_pred_adj = RepeatVector(100)(coarse_pred)
    coarse_pred_adj = Flatten()(coarse_pred_adj)
    prob_ave = Multiply()([fine_to_coarse, coarse_pred_adj])

    return prob_ave

def calc_threshold(fine_pred, fine_to_coarse, shared_layer):
    coarse_pred = Dot(1)([fine_pred, fine_to_coarse])
    coarse_pred_thresholded = Threshold()(coarse_pred)
    threshold_adj = RepeatVector(12*8*8)(coarse_pred_thresholded)
    threshold_adj = Reshape((8, 8, 12))(threshold_adj)
    shared_layer_adj = Multiply()([shared_layer, threshold_adj])
    
    return shared_layer_adj

In [13]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, Dense, Dropout, Multiply, Add, Dot, RepeatVector, Reshape
#from keras.layers.advanced_activations import relu
from keras.backend import constant, relu
from keras.models import Model

coarse_categories = 20
fine_categories = 100
sigma = .01

input_img = Input(shape=(32, 32, 3), dtype='float32', name='main_input')

## SHARED LAYERS
net_shared = Conv2D(24, (4, 4), strides=(1, 1), padding='same', activation='elu')(input_img)
net_shared = MaxPooling2D((2, 2), padding='same')(net_shared)
net_shared = Dropout(0.25)(net_shared)
net_shared = Conv2D(12, (2, 2), strides=(1, 1), padding='same', activation='elu')(net_shared)
net_shared = MaxPooling2D(pool_size=2, strides=2, padding='same')(net_shared)
shared_output = Dropout(0.25)(net_shared)

# COARSE CLASSIFIER
# Get the coarse predictions from fine
output_c = indep_layers(shared_output)

c = calc_inputs()

for i in range(20):
    thresholded = calc_threshold(output_c, c[i], shared_output)
    fine_output = indep_layers(thresholded)

    prob_ave = calc_prob_ave(output_c, c[i])

    weighted = Multiply()([fine_output, prob_ave])
    
    if i==0: output_f = weighted
    else: output_f = Add()([weighted, output_f])

model = Model(inputs=[input_img, c[0], c[1], c[2], c[3], c[4], c[5], c[6], c[7], c[8], c[9], c[10], c[11], c[12], c[13], c[14], c[15], c[16], c[17], c[18], c[19]], 
              outputs=output_f)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


**Fit Training Data**

In [14]:
ck = np.zeros((20,len(x_train),100))
for i in range(len(x_train)):
    for j in range(20):
        ck[j,i] = C2K_dot[j]

index= 0
step = 25

model.fit([x_train, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
          y_train_adj, epochs=step, batch_size=256, validation_split=.1)
model.save_weights('data/models/layer_'+str(index))

Train on 45000 samples, validate on 5000 samples
Epoch 1/25
45000/45000 [==============================] - 169s - loss: 0.1671 - acc: 0.0102 - val_loss: 1.1921e-07 - val_acc: 0.0084
Epoch 2/25
45000/45000 [==============================] - 154s - loss: 1.1921e-07 - acc: 0.0102 - val_loss: 1.1921e-07 - val_acc: 0.0084
Epoch 3/25
45000/45000 [==============================] - 153s - loss: 1.1921e-07 - acc: 0.0102 - val_loss: 1.1921e-07 - val_acc: 0.0084
Epoch 4/25
45000/45000 [==============================] - 153s - loss: 1.1921e-07 - acc: 0.0102 - val_loss: 1.1921e-07 - val_acc: 0.0084
Epoch 5/25
45000/45000 [==============================] - 153s - loss: 1.1921e-07 - acc: 0.0102 - val_loss: 1.1921e-07 - val_acc: 0.0084
Epoch 6/25
45000/45000 [==============================] - 153s - loss: 1.1921e-07 - acc: 0.0102 - val_loss: 1.1921e-07 - val_acc: 0.0084
Epoch 7/25
45000/45000 [==============================] - 152s - loss: 1.1921e-07 - acc: 0.0102 - val_loss: 1.1921e-07 - val_acc: 0.0

**Fit on dataset delayed (for use in browsers)**

This is a complex network and the dataset is not small either so this may occasionally crash if run in a browser. 

In the case that it does, use this function to incrementally train the network 5 (# can be changed if necessary), save the model and then reload it for the next increment of training.

In [22]:
while index < 300:
    index += step
    model.load_weights('data/models/layer_'+str(index-5), by_name=False)
    model.fit([x_train, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
              y_train_adj, epochs=index+step, batch_size=64, initial_epoch=index, validation_split=.1)
    model.save_weights('data/models/layer_'+str(index))

Train on 45000 samples, validate on 5000 samples
Epoch 31/50
45000/45000 [==============================] - 49s - loss: 3.2864 - acc: 0.2065 - val_loss: 3.3163 - val_acc: 0.2038
Epoch 32/50
45000/45000 [==============================] - 69s - loss: 3.2859 - acc: 0.2023 - val_loss: 3.2988 - val_acc: 0.2126
Epoch 33/50
45000/45000 [==============================] - 80s - loss: 3.2738 - acc: 0.2055 - val_loss: 3.3011 - val_acc: 0.2078
Epoch 34/50
45000/45000 [==============================] - 49s - loss: 3.2796 - acc: 0.2030 - val_loss: 3.4280 - val_acc: 0.1874
Epoch 35/50
45000/45000 [==============================] - 47s - loss: 3.2689 - acc: 0.2062 - val_loss: 3.2695 - val_acc: 0.2108
Epoch 36/50
45000/45000 [==============================] - 52s - loss: 3.2595 - acc: 0.2053 - val_loss: 3.2568 - val_acc: 0.2138
Epoch 37/50
45000/45000 [==============================] - 50s - loss: 3.2544 - acc: 0.2094 - val_loss: 3.2536 - val_acc: 0.2162
Epoch 38/50
45000/45000 [=======================

In [17]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

**Evaluate on testing set**

In [ ]:
ct = np.zeros((20,len(x_test),100))
for i in range(len(x_test)):
    for j in range(20):
        ct[j,i] = C2K_dot[j]
        
model.evaluate([x_test, ck[0], ck[1], ck[2], ck[3], ck[4], ck[5], ck[6], ck[7], ck[8], ck[9], ck[10], ck[11], ck[12], ck[13], ck[14], ck[15], ck[16], ck[17], ck[18], ck[19]],\
          y_test_adj, batch_size=64, verbose=1)